### Set Up

In [ ]:
import findspark 
findspark.init() 

import pyspark
from delta import *
import pandas as pd
from pyspark.sql.functions import col

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
spark = pyspark.sql.SparkSession.builder.master("spark://spark-master:7077").appName("spark_kafka").config("spark.sql.streaming.metricsEnabled", "true").getOrCreate()

sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [ ]:
import sys
sys.path.append('/opt/spark/work-dir/odp_intra_storage/ingestion_framework/')

### Bronze Metadata

In [ ]:
spark.sql("create database if not exists bronze").toPandas()

In [ ]:
spark.sql("show databases").toPandas()

In [ ]:
from metadata_refresh.bronze_refresh import refresh_bronze_metadata

In [ ]:
env = 'test'
bronze_metadata_file_location = '/opt/spark/work-dir/odp_intra_storage/vysh_temp/spark_clients/notebooks/bronze/bronze_metadata.csv'
bronze_metadata_table_name = 'bronze.metadata_table'

spark.sql(f"drop table if exists {bronze_metadata_table_name}")

refresh_bronze_metadata(spark, env, bronze_metadata_file_location, bronze_metadata_table_name)

In [ ]:
spark.table("bronze.metadata_table").where("pipeline_id = 1").select("*").toPandas()

### Streaming Pipeline

In [ ]:
from spark_ingestion_app.spark_ingestion_app import SparkIngestionApp

In [ ]:
# spark.sql("drop table if exists bronze.weather_test_v1").toPandas()

In [ ]:
module_parameters = '\'{"group_id": "1", "pipeline_id": "1"}\''

SparkIngestionApp.run_module(spark = spark, module_name = 'KafkaToBronze', module_parameters = module_parameters, log_level='INFO')

In [ ]:
spark.sql("select * from bronze.weather_test_v1 limit 10").toPandas()

In [ ]:
spark.sql("select current_timestamp, max(timestamp) from bronze.weather_test_v1 limit 10").toPandas()